In [26]:
import pandas as pd
import requests
from bs4 import BeautifulSoup  as BS # Импортируем библиотеку BeautifulSoup
from fake_useragent import UserAgent # для обхода блокировки парсинга сайтом
import pandas as pd
from time import sleep
import re

In [127]:


BASE_URL = 'https://reflexo.io/ru/tcm/acupuncture/points' # Определяем значение URL страницы для запроса
HEADERS = {'User-Agent': UserAgent().random} # генерируем рандомный UserAgent для сайта

response = requests.get(BASE_URL, headers=HEADERS) # в сессии делаем запрос
soup = BS(response.text, 'html.parser') # Создаём объект BeautifulSoup с содержимым r, указывая html-парсер
links = soup.find('div', class_='border list-group lg-points').text.split(' | ')[1:]

points = pd.DataFrame(links, columns=['points'])
points
points.points = points.points.str.extract(r"(\w*)")
for a in points.points.index:
    points.loc[a, 'points'] = re.sub(r'(\d+)', '', points.loc[a, 'points'])
points

In [140]:
points.points[:3].to_list()

['Jingming', 'Zanzhu', 'Meichong']

In [158]:
pointt = []
names = []
znachs = []
locls = []
per = []

for p in points.points.to_list():
    response = requests.get(BASE_URL + '/' + p, headers=HEADERS) # в сессии делаем запрос
    soup = BS(response.text, 'html.parser') # Создаём объект BeautifulSoup с содержимым r, указывая html-парсер
    point = soup.find('h1', class_='smallHeading d-inline').text.strip()
    pointt.append(point)
    name = soup.find('h1', class_='smallHeading d-inline', id='nameEngPinin').text.strip()
    names.append(name)
    zn = soup.find_all('p', class_='mx-3')#.text.strip()
    znach = zn[0].text
    znachs.append(znach)
    local = zn[1].text
    locls.append(local)
    peresech = zn[3].text
    per.append(peresech)

points_data = pd.DataFrame(
    {
        "Точка":pointt,
        "Название":names,
        "Значение":znachs,
        "Локализация":locls,
        "Пересечение":per
    }
)

In [230]:
points_data["Точка"] = points_data["Точка"].str.replace('RenMai','Cv')#, 'Gb':'Gb', 'St':'St', 'Du':'DuMai', 'KD':'Kid', 'Ren':'RenMai', 'SJ':'Th', 'LI':'Co', 'SP':'Sp', 'SI':'Si', 'Liv':'Liv','LU':'Lu', 'HT':'Ht', 'Pc':'Hg'})


In [235]:
points_data

,Точка,Название,Значение,Локализация
0,Bl1,Jingming,Цзин» (目胃)一 глаза; «минг» 一 просветление. Точ...,"На 0,1 цуня кнутри от внутреннего угла глаза."
1,Bl2,Zanzhu,Цуань — «сходиться вместе»; «чжу» (竹)—«бамбук...,"В углублении у внутреннего края брови, прямо в..."
2,Bl3,Meichong,Мэй — «бровь»; «чун» — «устремляться». В книг...,"Прямо вверх от внутреннего края брови, в серед..."
3,Bl4,Quchai,Цюй (曲)——изгиб; «чай» (差)一отклоняться в сторо...,"На 1,5 цуня кнаружи от точки Шэнь-тин VG.24"
4,Bl5,Wuchu,"У (五)——«пять, пятый»; чу (处)—«место». Точка У...","Прямо вверх от точки Цюй-чай V.4, на 1 цунь вы..."
...,...,...,...,...
356,St41,Jiexi,"1. «Цзе» (解)——«сустав, сочленение костей»; «си...",На тыле стопы в центре поперечной складки голе...
357,St42,Chongyang,"«Чун» — импульс, толчок,биение; «ян» —Ян-кана,...","На самой возвышенной части тыла стопы, между с..."
358,St43,Xiangu,«Сянь» — «впадина»; «гу» — «ущелье». Точка рас...,В межкостной щели между II и III плюсневыми ко...
359,St44,Neiting,Нэй» (内)—«внутри»; «тин» (庭)—«внутренний двор ...,"На тыле стопы, на конце щели между II и III па..."


In [236]:
points_data.to_csv("points.csv")

In [ ]:
# {"Канал":"Gb","Liv","Lu","Co" }
# {"Симптом":"Боль в височных областях"}